<a href="https://colab.research.google.com/github/framunoz/cv-analyser-with-rag/blob/feature%2Fkaggle-colab-notebooks/notebooks/colab-ai-driven-cv-optimisation-with-rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Optimize Your CV with AI to Beat the ATS Filters

## 1. Overview

**What does this notebook do?**

Applying for jobs often means getting past automated screening software (called ATS - Applicant Tracking Systems) *before* a human reads your CV. This notebook acts as your **personal AI assistant** to help tailor your CV for specific job openings, increasing your chances of getting noticed.

Here's the plan:
1.  You provide your CV and the description of a job you want.
2.  AI reads and understands your CV's content.
3.  It compares your CV to the job description to find your **most relevant** experiences or skills for that role.
4.  Then, the AI helps you **rewrite** those key parts of your CV, adding important keywords from the job description to make it **ATS-friendly** and more impactful for recruiters.

**Who is this for?**

Anyone applying for jobs who wants a smarter way to customize their CV for each application. **No AI or programming knowledge is needed!**

**How it works (in simple terms):**

* **Understanding Text:** The AI reads the text from your CV.
* **Making Connections (Embeddings & RAG):** It represents the meaning of your CV sections and the job description in a special way (like giving ideas coordinates). This allows it to mathematically find which parts of your CV are the closest match to the job requirements (this smart searching is sometimes called RAG).
* **AI Writing Assistant (LLM):** It uses a powerful AI language model (Google's Gemini) to help rephrase your relevant experiences, weaving in keywords naturally.
* **You're in Control:** You'll interact with the AI in a chat to guide the rewriting process, ensuring the final text sounds like you.

**Steps We'll Follow:**

1.  **Setup:** Prepare the notebook (install tools, add your secure Google AI API key).
2.  **Your Input:** Provide your CV and the job description using simple forms/buttons.
3.  **CV Processing:** AI reads and structures your CV information.
4.  **Create CV "Memory":** AI creates a searchable representation of your CV content (using embeddings and a vector database called ChromaDB).
5.  **Find Best Matches:** AI searches its "memory" to find the parts of your CV most relevant to the job.
6.  **Rewrite & Refine:** You chat with the AI to optimize the suggested CV sections.
7.  **Review:** Check the final optimized text.

## 2. Setup

This section handles the initial preparation needed to run the notebook in Google Colab. We'll install necessary software libraries and guide you on securely setting up your Google AI API key.

### 2.1. Dependencies

The first step is to install the necessary Python libraries. These are software packages that give our notebook the tools needed to work with PDFs, connect to the Google AI services, manage the special "CV memory" database (ChromaDB), and handle file uploads. This cell might take a minute or two to run.

In [1]:
#@title Install Required Libraries (click the ► button to run)
# This command installs the necessary Python packages for the notebook.
# It might take a minute or two to complete.
# The '-q' flag makes the output less noisy (quieter).

# Uninstall conflicting library (optional step from original notebook, may help in Colab)
!pip uninstall -qqy jupyterlab

# Install Google AI, PDF reader, Vector DB, GDrive downloader (if used), YAML handler
# We install a specific version of google-genai used in the original development
# and typing_extensions for compatibility.
!pip install -q google-genai pdfplumber chromadb gdown PyYAML typing_extensions

print("✅ Libraries installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:0

### 2.2. API Key Configuration (Colab Secrets)

To use the Google AI services (like Gemini), this notebook needs your API key. Colab offers convenient and secure ways to add it using the **Secrets** manager (click the **🔑 key icon** in the left sidebar).

**➡️ Providing Your API Key (Choose ONE method):**

* **Method 1: Import from Google AI Studio (Recommended)**
    1.  Click the **🔑 (key) icon** in the left sidebar to open Secrets.
    2.  Find the **"Gemini API keys"** dropdown menu.
    3.  Select **"Import key from Google AI Studio"**.
    4.  Follow the on-screen prompts to authenticate and import your existing key. Colab should store it securely, likely naming it `GOOGLE_API_KEY` automatically.

* **Method 2: Add Manually**
    1.  Click the **🔑 (key) icon** in the left sidebar.
    2.  Click **"+ Add new secret"**.
    3.  Enter the **Name** exactly as `GOOGLE_API_KEY`.
    4.  Paste your Google AI API key (obtainable from [Google AI Studio](https://aistudio.google.com/app/apikey) or Google Cloud) into the **Value** field.
    5.  **Enable** the toggle switch ☑️ "Notebook access".
    6.  Close the Secrets panel.

The next code cell will attempt to read the secret named `GOOGLE_API_KEY`. **Ensure the key is available via one of these methods before running the next cell.**

In [2]:
#@title Load Google AI API Key from Colab Secrets (click the ► button to run)
from google.colab import userdata

# This dictionary provides helpful messages based on errors
_API_KEY_ERROR_MESSAGES = {
    "NOT_FOUND": """
❌ **Secret 'GOOGLE_API_KEY' not found.**
   Please ensure you have added the secret correctly using one of the methods
   described above (using the 🔑 icon in the left sidebar) and that
   'Notebook access' is enabled for it.
    """,
    "EMPTY": """
❌ **Secret 'GOOGLE_API_KEY' was found but is empty.**
   Please check the secret value you added using the 🔑 icon in the left sidebar.
    """,
    "IMPORT_ERROR": """
❌ **Could not access Colab Secrets.**
   This notebook is designed to run in Google Colab. If you are in Colab,
   try restarting the runtime (Runtime -> Restart runtime).
    """,
    "UNKNOWN": """
❌ **An unexpected error occurred while loading the API key.**
   Details: {error}
    """
}

try:
    # Attempt to retrieve the secret named 'GOOGLE_API_KEY'
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    if GOOGLE_API_KEY is None:
        # Secret not found or access not enabled
        print(_API_KEY_ERROR_MESSAGES["NOT_FOUND"])
        raise ValueError("API Key not found.") # Stop execution
    elif GOOGLE_API_KEY == "":
        # Secret found but has no value
        print(_API_KEY_ERROR_MESSAGES["EMPTY"])
        raise ValueError("API Key is empty.") # Stop execution
    else:
        # Key loaded successfully
        print("✅ GOOGLE_API_KEY loaded successfully from Colab Secrets.")
        # Optional: Uncomment below to verify the first few chars (for debugging)
        # print(f"(Key starts with: {GOOGLE_API_KEY[:4]}...)")

except ImportError:
    # Handle case where userdata is not available (running outside Colab)
    print(_API_KEY_ERROR_MESSAGES["IMPORT_ERROR"])
    GOOGLE_API_KEY = None
    raise # Stop execution
except Exception as e:
    # Handle any other unexpected errors during secret retrieval
    print(_API_KEY_ERROR_MESSAGES["UNKNOWN"].format(error=e))
    GOOGLE_API_KEY = None
    raise # Stop execution

# If the code reaches here without raising an error, the key is loaded.

✅ GOOGLE_API_KEY loaded successfully from Colab Secrets.


### 2.3. Helper Functions (Retry Logic)

Sometimes, when contacting the Google AI service, it might be temporarily busy or unavailable (like getting a busy signal on a phone line). To handle this smoothly, the next code cell defines a small helper function.

This helper allows the notebook to automatically try contacting the service again a few times if it encounters these specific temporary issues. This makes the whole process more reliable without you having to manually re-run things for minor glitches. You don't need to worry about the details of the code itself.

In [4]:
#@title Define Helper Function for API Retries (click to expand)
from google import genai
from google.api_core import retry

def is_retriable(exception: Exception) -> bool:
    """Checks if an exception is a known temporary/retriable Google API Error."""
    return isinstance(exception, genai.errors.APIError) and exception.code in {429, 503}

## 3. User Configuration

Adjust the variables in the following code cell to control the notebook's behavior. These include file paths, language settings, model parameters, and the job description you want to analyze.

In [9]:
from pathlib import Path
from typing import Literal

# ==============================================================================
# User Configuration Variables
# ==============================================================================
# Adjust these settings before running the rest of the notebook.

# --- 1. Input Files & Paths ---

# Specify the path where the CV PDF file is located or will be downloaded to.
CV_PDF_PATH = Path("./resume.pdf")  # Example: file in the root working directory

# Set to True to download the CV from Google Drive using the ID below.
# Set to False if you are providing the CV via CV_PDF_PATH directly.
DOWNLOAD_CV = True  # Default behavior from original notebook
CV_GDRIVE_FILE_ID = "1avK0u9HcyuEYgpyIs_pBuzPxWPrfgW_C"  # Example File ID

# --- 2. Job Description Input ---

# Paste the target job description text between the triple quotes.
JOB_DESCRIPTION = """
Job Title: AI/Machine Learning Engineer

Company: Innovate Solutions Inc.

Location: Remote (US Based)

About Us:
Innovate Solutions Inc. is at the forefront of applying artificial intelligence to solve real-world business challenges. We foster a collaborative environment where creative thinking and technical excellence drive our success. We are passionate about building intelligent systems that deliver significant value to our clients across various industries. Join our growing team and help shape the future of applied AI.

About the Role:
We are seeking a talented and motivated AI/Machine Learning Engineer to join our core development team. You will play a key role in the end-to-end lifecycle of machine learning projects, from conceptualization and data exploration to model deployment and monitoring. You'll work closely with data scientists, software engineers, and product managers to build innovative AI-powered features and products.

Responsibilities:
- Design, develop, train, and deploy machine learning models (including deep learning models) for tasks such as NLP, predictive analytics, anomaly detection, and personalization.
- Process, cleanse, and verify the integrity of large datasets used for analysis and model training.
- Collaborate with data engineering teams to build and maintain robust data pipelines for ML workflows.
- Implement and maintain MLOps best practices for model versioning, testing, deployment, and monitoring.
- Stay current with the latest advancements in AI/ML techniques, tools, and platforms.
- Analyze experimental results, iterate on models, and communicate findings to technical and non-technical stakeholders.
- Contribute to the development of internal AI platforms and tooling.

Required Qualifications:
- Bachelor's or Master's degree in Computer Science, Data Science, Statistics, or a related quantitative field.
- 2+ years of hands-on experience building and deploying machine learning models in a production environment.
- Strong programming skills in Python and proficiency with relevant ML libraries (e.g., Scikit-learn, TensorFlow, PyTorch, Keras).
- Solid understanding of core machine learning algorithms, statistical modeling, and evaluation metrics.
- Experience working with SQL and/or NoSQL databases.
- Familiarity with data processing and analysis libraries (e.g., Pandas, NumPy).
- Excellent problem-solving skills and attention to detail.
- Strong communication and teamwork abilities.

Desired Qualifications (Bonus Points):
- PhD in a related field.
- Experience with cloud platforms (AWS, GCP, or Azure) and their AI/ML services (e.g., SageMaker, Vertex AI, Azure ML).
- Experience with MLOps tools and practices (e.g., Docker, Kubernetes, MLflow, Kubeflow).
- Experience with Natural Language Processing (NLP) or Computer Vision (CV).
- Experience with big data technologies (e.g., Spark, Hadoop).
- Publications in relevant AI/ML conferences or journals.

What We Offer:
- Competitive salary and benefits package.
- Opportunity to work on challenging and impactful AI projects.
- A dynamic, collaborative, and supportive work environment.
- Flexible remote work policy.
- Professional development opportunities.
"""

# --- 3. Language Settings ---

# Set the primary language for prompts (influences LLM responses).
LANGUAGE: Literal["en", "es"] = "en"  # Options: 'en' or 'es'

# --- 4. RAG & Embedding Settings ---

# Retrieval Parameters for RAG Search (Controls how many results are fetched/processed)
MAX_RELEVANT_ITEMS = 3  # Target number of CV items to refine
RETRIEVAL_WINDOW = 2  # Extra items retrieved for context during search

# Embedding Model Name
EMBEDDING_MODEL_NAME = "models/text-embedding-004"

# Vector Database Configuration (Using Single Collection Strategy)
CHROMA_DB_PATH = Path("./chroma_db_persistent")  # Path for persistent DB
COLLECTION_NAME = "cv_embeddings_v1"  # Name for the single collection

# --- 5. LLM Configuration ---

# Generative model name for structuring the CV and for the rewriting chat.
# Using the specific name mentioned by the user.
GENERATIVE_MODEL_NAME = "gemini-2.0-flash"

# Parameters for the PDF -> JSON *Structuring* LLM call
STRUCTURING_LLM_TEMPERATURE = 0.1

# Parameters for the CV Item *Rewriting* LLM Chat
REWRITING_LLM_TEMPERATURE = 0.8
REWRITING_LLM_TOP_P = 0.95
REWRITING_LLM_TOP_K = 30

# --- 6. CV Processing Settings ---

# List the sections (JSON keys) from the structured CV to embed and use for RAG.
# Keeping this as a list allows processing/searching multiple sections with the single DB collection.
# Example sections based on JSON Resume schema and original recommendations:
# "work", "certificates", "publications", "projects", "skills", "education"
CV_SECTIONS_TO_FOCUS: list[str] = [
    "work",
    "certificates",
    "projects",
]  # Edit this list as needed

# Optional: Limit characters fed to the structuring LLM. None = no limit.
MAX_CV_TEXT_LENGTH_FOR_STRUCTURING = 12000


# ==============================================================================
# Directory Setup (Using pathlib)
# ==============================================================================

# Ensure the directory for ChromaDB exists
CHROMA_DB_PATH.mkdir(parents=True, exist_ok=True)

# Ensure the parent directory for the CV PDF exists
CV_PDF_PATH.parent.mkdir(parents=True, exist_ok=True)


print("User configuration loaded and directories created.")

## 4. Load and Process CV

### 4.1. Load PDF Document

This step ensures the CV PDF file specified in the configuration (`CV_PDF_PATH`) is available. If `DOWNLOAD_CV` was set to `True`, it downloads the file from Google Drive using the provided ID. Otherwise, it assumes the file exists at `CV_PDF_PATH`.

Finally, it checks if the file exists and displays the PDF inline for verification.

In [20]:
import gdown
from IPython.display import IFrame, display

# --- 1. Conditional Download ---
if DOWNLOAD_CV:
    print(
        f"Attempting to download CV from Google Drive (ID: {CV_GDRIVE_FILE_ID}) to"
        f" '{CV_PDF_PATH}'..."
    )
    try:
        # Download using gdown, specifying the output path
        gdown.download(id=CV_GDRIVE_FILE_ID, output=str(CV_PDF_PATH), quiet=False)
        print("Download attempt finished.")
    except Exception as e:
        print("\nERROR: Failed to download file from Google Drive.")
        print(
            f"Please check the file ID ('{CV_GDRIVE_FILE_ID}') and ensure the file is"
            " accessible."
        )
        print(f"Error details: {e}")
else:
    print(f"Skipping download. Assuming CV PDF is already present at: '{CV_PDF_PATH}'")

# --- 2. Verify File Existence and Display ---
if CV_PDF_PATH.is_file():
    print(f"CV PDF found at '{CV_PDF_PATH}'. Displaying PDF inline below:")
    # Display the PDF inline for verification
    display(IFrame(src=CV_PDF_PATH, width="90%", height="600px"))
else:
    print(f"\nERROR: CV PDF file was not found at the path: '{CV_PDF_PATH}'")
    # Provide context based on whether download was attempted
    if DOWNLOAD_CV:
        print("The download may have failed, or the configured path is incorrect.")
    else:
        print(
            "Please ensure the file exists at the specified path in the configuration."
        )

NameError: CV Schema ('Curriculum') not found.

### 4.2. Extract Text from PDF

Now that the PDF file is available, we will extract its text content. We use the `pdfplumber` library for this task, as it's generally effective for text-based PDFs. The extracted text will be stored in a variable for the next step (structuring with the LLM).

In [ ]:
import pdfplumber

raw_cv_text = ""  # Initialize variable

print(f"Extracting text from PDF: '{CV_PDF_PATH}'...")
try:
    with pdfplumber.open(CV_PDF_PATH) as pdf:
        # Extract text page by page, handling potential None values from empty pages
        raw_cv_text = "".join(
            page.extract_text(x_tolerance=1, y_tolerance=3) or "" for page in pdf.pages
        )

    print(f"Text extraction successful. Total characters: {len(raw_cv_text)}")
    # Optional: Uncomment the line below to print the first 500 characters
    print(f"--- Snippet ---\n{raw_cv_text[:500]}\n---------------")

except FileNotFoundError:
    print(f"\nERROR: PDF file not found at '{CV_PDF_PATH}'. Cannot extract text.")
    raise
except Exception as e:
    print(f"\nERROR: Failed to open or extract text from PDF '{CV_PDF_PATH}'.")
    print(f"Error details: {e}")
    raise

### 4.3. Structure Text into JSON using LLM

In this crucial step, we leverage the configured Generative AI model (Gemini) to parse the unstructured `raw_cv_text` extracted from the PDF. We instruct the model to return the information structured according to the [JSON Resume schema](https://jsonresume.org/schema/).

This involves:
1.  Defining the JSON Resume schema using Python's `TypedDict` for type safety and clarity (this helps the model adhere to the desired output structure).
2.  Constructing a prompt that tells the model the task, provides the raw CV text (potentially truncated based on `MAX_CV_TEXT_LENGTH_FOR_STRUCTURING`), and specifies the desired JSON output format.
3.  Calling the Gemini API, explicitly requesting JSON output and providing the schema definition.
4.  Parsing the LLM's JSON response into a Python dictionary.

In [ ]:
from typing_extensions import TypedDict


# Define nested structures first (order matters for definition)
class Location(TypedDict, total=False):
    address: str
    postalCode: str
    city: str
    countryCode: str
    region: str


class Profile(TypedDict, total=False):
    network: str
    username: str
    url: str


class Basics(TypedDict, total=False):
    name: str
    label: str
    image: str  # URL to image
    email: str
    phone: str
    url: str  # Personal website/portfolio URL
    summary: str  # Professional summary
    location: Location
    profiles: list[Profile]  # Use modern list hint


class WorkItem(TypedDict, total=False):
    name: str  # Name of the company/organization
    position: str  # Job title
    url: str  # Company website
    startDate: str  # Format YYYY-MM-DD or YYYY-MM or YYYY
    endDate: str  # Format YYYY-MM-DD or YYYY-MM or YYYY, or Present
    summary: str  # High-level description of role/company
    highlights: list[str]  # Specific achievements or responsibilities (bullet points)


class VolunteerItem(TypedDict, total=False):
    organization: str
    position: str
    url: str
    startDate: str
    endDate: str
    summary: str
    highlights: list[str]


class EducationItem(TypedDict, total=False):
    institution: str
    url: str
    area: str  # e.g., Computer Science
    studyType: str  # e.g., Bachelor's Degree, Master's
    startDate: str
    endDate: str
    score: str  # e.g., GPA
    courses: list[str]  # Relevant coursework


class AwardItem(TypedDict, total=False):
    title: str
    date: str  # Date awarded
    awarder: str  # Organization that gave the award
    summary: str  # Description of the award


class CertificateItem(TypedDict, total=False):
    name: str  # Name of the certificate
    date: str  # Date issued
    issuer: str  # Issuing organization (e.g., Coursera, Google)
    url: str  # Link to certificate if available


class PublicationItem(TypedDict, total=False):
    name: str  # Title of the publication
    publisher: str  # e.g., Journal name, Conference
    releaseDate: str
    url: str  # Link to publication
    summary: str  # Abstract or brief description


class SkillItem(TypedDict, total=False):
    name: str  # Broad skill category (e.g., Web Development, Data Science)
    level: str  # Optional proficiency level (e.g., Intermediate, Advanced)
    keywords: list[str]  # Specific technologies or tools (e.g., Python, PyTorch, AWS)


class LanguageItem(TypedDict, total=False):
    language: str  # e.g., English, Spanish
    fluency: str  # e.g., Native, Fluent, Conversational


class InterestItem(TypedDict, total=False):
    name: str  # Category of interest (e.g., Open Source, AI Ethics)
    keywords: list[str]  # Specific interests


class ReferenceItem(TypedDict, total=False):
    name: str  # Name of reference (ensure consent)
    reference: str  # Testimonial or contact details (handle privacy appropriately)


class ProjectItem(TypedDict, total=False):
    name: str  # Project title
    startDate: str
    endDate: str
    description: str  # Overall description of the project
    highlights: list[str]  # Key contributions or features
    url: str  # Link to project demo or repository


# --- Top-Level Curriculum Schema ---
class Curriculum(TypedDict, total=False):
    """Represents the complete JSON Resume structure."""

    basics: Basics
    work: list[WorkItem]
    volunteer: list[VolunteerItem]
    education: list[EducationItem]
    awards: list[AwardItem]
    certificates: list[CertificateItem]
    publications: list[PublicationItem]
    skills: list[SkillItem]
    languages: list[LanguageItem]
    interests: list[InterestItem]
    references: list[ReferenceItem]
    projects: list[ProjectItem]

In [ ]:
import json
from google import genai
from google.genai import types
from google.api_core import retry

# --- 1. Initialize Model & Validate API Key ---
if "GOOGLE_API_KEY" not in locals() or not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found or empty. Please check Kaggle Secrets.")

client = genai.Client(api_key=GOOGLE_API_KEY)

# --- 2. Prepare Prompt & Configuration for JSON Structuring ---
cv_text_for_prompt = raw_cv_text
if (
    MAX_CV_TEXT_LENGTH_FOR_STRUCTURING
    and len(raw_cv_text) > MAX_CV_TEXT_LENGTH_FOR_STRUCTURING
):
    cv_text_for_prompt = raw_cv_text[:MAX_CV_TEXT_LENGTH_FOR_STRUCTURING]
    print(
        f"Warning: CV text truncated to {MAX_CV_TEXT_LENGTH_FOR_STRUCTURING} chars for"
        " structuring prompt."
    )

# System instruction defines the LLM's role and desired output format
system_instruction = (
    "You are an expert CV parser. Extract information from the provided CV text and"
    " format it strictly according to the JSON Resume Schema provided. Return ONLY the"
    " valid JSON object conforming to the schema - no introductory text, no markdown"
    " formatting ('```json', '```'), no explanations."
)
# Prompt combines instructions with the actual CV text
prompt_message = f"""
Given the following CV text, populate the fields of the JSON Resume Schema as accurately as possible.
Use empty strings, arrays, or null values for fields where information is missing in the text.

CV Text:
---
{cv_text_for_prompt}
---
"""
full_structuring_prompt = system_instruction + "\n\n" + prompt_message

# Configuration forces JSON output matching our TypedDict schema
json_generation_config = {
    "temperature": STRUCTURING_LLM_TEMPERATURE,
    "response_mime_type": "application/json",
    "response_schema": Curriculum,
}


# --- 3. Define Function for API Call with Retry Logic ---
@retry.Retry(predicate=is_retriable)  # Use helper defined in Setup
def generate_structured_cv_json_with_retry(prompt, config):
    """Calls the Gemini API to generate structured JSON, with retries on specific errors."""
    print("Calling Gemini API to structure CV text into JSON...")
    response = client.models.generate_content(
        model=GENERATIVE_MODEL_NAME,
        contents=prompt,
        config=config,
    )
    print("Gemini API call finished.")
    return response.text


# --- 4. Execute API Call and Parse Response ---
structured_cv_data = None  # Initialize result variable

try:
    json_response_text = generate_structured_cv_json_with_retry(
        prompt=full_structuring_prompt,
        config=json_generation_config,
    )

    if json_response_text:
        try:
            # Parse the validated JSON text from the API response
            structured_cv_data = json.loads(json_response_text)
            print("Successfully parsed LLM response into structured CV data.")
        except json.JSONDecodeError as e:
            # This error *should* be rare given response_mime_type="application/json"
            print("\nERROR: Failed to parse the LLM's response as JSON.")
            print(f"JSONDecodeError: {e}")
            print("\nLLM Response Text Received:\n---\n", json_response_text, "\n---")
    else:
        print("\nERROR: Received an empty response from the LLM API.")

except Exception as e:
    print("\nERROR: An unexpected error occurred during the LLM structuring call.")
    print(f"Error details: {e}")

### 4.4. (Optional) Display Structured CV Data

You can run the next cell to print the `structured_cv_data` dictionary in a readable YAML format. This helps verify that the LLM correctly parsed and structured the information from your CV text according to the JSON Resume schema. Check if key sections like 'basics', 'work', 'education', 'skills', etc., were populated as expected.

In [ ]:
import yaml

print("#" + "=" * 79)
print("# Structured CV Data (YAML Format):")
print("#" + "=" * 79)
print(
    yaml.dump(
        structured_cv_data, allow_unicode=True, sort_keys=False, width=float("inf")
    )
)
print("#" + "=" * 79)

## 5. Prepare and Store Embeddings in ChromaDB

This section focuses on converting the relevant parts of your structured CV data into vector embeddings and storing them in a searchable vector database (ChromaDB). This allows us to later find the CV items most semantically similar to a job description (the core of RAG).

The key steps are:
1.  **Prepare Documents:** Extract the specific CV items (like individual work experiences or projects) from the sections listed in `CV_SECTIONS_TO_FOCUS` and format them as text documents suitable for embedding. We'll use the YAML dump method as previously decided.
2.  **Define Embedding Function:** Set up a function that uses the configured Google embedding model (`EMBEDDING_MODEL_NAME`) to convert text documents into numerical vectors (embeddings).
3.  **Initialize Vector Database:** Connect to or create a persistent ChromaDB database using the configured path (`CHROMA_DB_PATH`) and collection name (`COLLECTION_NAME`). We will use a single collection for all items.
4.  **Generate & Store Embeddings:** Iterate through the prepared documents, generate embeddings for them using the embedding function, and add the embeddings, the original documents, and relevant metadata (like the source section) to the ChromaDB collection.

### 5.1. Prepare Text Documents from Structured CV

In [ ]:
import yaml
import re

# --- 1. Helper Functions for ID Generation ---

# Maps section keys to functions creating a base ID string from item content
BASE_ID_GENERATORS = {
    "work": lambda item: (
        f"{item.get('name', 'NoCompany')}.{item.get('position', 'NoPosition')}"
    ),
    "certificates": lambda item: (
        f"{item.get('issuer', 'NoIssuer')}.{item.get('name', 'NoCert')}"
    ),
    "publications": lambda item: (
        f"{item.get('publisher', 'NoPublisher')}.{item.get('name', 'NoPub')}"
    ),
    "projects": lambda item: item.get("name", "NoProject"),
    "volunteer": lambda item: (
        f"{item.get('organization', 'NoOrg')}.{item.get('position', 'NoVolunteerPos')}"
    ),
    "education": lambda item: (
        f"{item.get('institution', 'NoInstitution')}.{item.get('area', 'NoArea')}.{item.get('studyType', '')}"
    ),
    "basics": lambda item: item.get("name", "NoPerson"),
    "awards": lambda item: (
        f"{item.get('awarder', 'NoAwarder')}.{item.get('title', 'NoAward')}"
    ),
    "skills": lambda item: item.get("name", "NoSkill"),
    "languages": lambda item: item.get("language", "NoLang"),
    "interests": lambda item: item.get("name", "NoInterest"),
    "references": lambda item: item.get("name", "NoReference"),
}


def sanitize_id(text_id: str) -> str:
    """Cleans and formats a string into a valid ChromaDB ID."""
    text_id = text_id.lower()
    # Basic accent removal using a lambda for compactness
    # fmt: off
    accent_map = {
        'á':'a', 'ä':'a', 'â':'a', 'à':'a', 'ã':'a', 'å':'a', 'é':'e', 'ë':'e',
        'ê':'e', 'è':'e', 'í':'i', 'ï':'i', 'î':'i', 'ì':'i', 'ó':'o', 'ö':'o',
        'ô':'o', 'ò':'o', 'õ':'o', 'ø':'o', 'ú':'u', 'ü':'u', 'û':'u', 'ù':'u',
        'ñ':'n', 'ç':'c',
    }
    # fmt: on
    text_id = re.sub(
        r"[áäâàãåéëêèíïîìóöôòõøúüûùñç]", lambda m: accent_map.get(m.group(0)), text_id
    )
    text_id = re.sub(r"[\s_:-]+", ".", text_id)  # Replace separators with dot
    text_id = re.sub(r"[^a-z0-9.]", "", text_id)  # Keep only alphanumeric and dot
    text_id = re.sub(r"\.+", ".", text_id)  # Consolidate consecutive dots
    text_id = text_id.strip(".")  # Remove leading/trailing dots
    if len(text_id) < 3:  # noqa: PLR2004
        text_id = f"{text_id}.id"  # Ensure min length
    return text_id[:63]  # Truncate to 63 characters (ChromaDB limit)


def generate_unique_item_id(section_key: str, item: dict, item_index: int) -> str:
    """Generates a unique, sanitized ID for a CV item using dot separator."""
    id_generator = BASE_ID_GENERATORS.get(
        section_key, lambda i: f"item.{item_index}"
    )  # Fallback
    base_id = id_generator(item)
    full_id_base = f"{section_key}.{base_id}.{item_index}"  # Use dot separator
    return sanitize_id(full_id_base)


# --- 2. Function to Prepare Data for Embedding ---


def prepare_embedding_data(
    cv_data: dict, sections_to_include: list[str]
) -> tuple[list, list, list]:
    """
    Extracts items, formats documents (YAML strings), generates IDs, creates metadata.
    Returns tuple: (documents, ids, metadatas).
    """
    all_documents = []
    all_ids = []
    all_metadatas = []

    if not isinstance(cv_data, dict):
        print("Warning: structured_cv_data is not a dictionary.")
        return [], [], []

    print(f"Preparing embedding data for sections: {sections_to_include}...")
    processed_count = 0
    for section_key in sections_to_include:
        section_items = cv_data.get(section_key)

        if not isinstance(section_items, list):
            print("Warning: Expected a list for section items.")
            continue

        for index, item in enumerate(section_items):
            if not isinstance(item, dict):
                print(
                    f"Warning: Expected a dictionary for item {index} in {section_key}."
                )
                continue

            item_id = generate_unique_item_id(section_key, item, index)
            try:
                item_doc = yaml.dump(
                    item,
                    allow_unicode=True,
                    sort_keys=False,
                    width=float("inf"),
                    default_flow_style=None,
                )
            except yaml.YAMLError:
                item_doc = str(item)  # Fallback
                print(f"Warning: YAML dump failed for item {index} in {section_key}.")

            metadata = {"section": section_key, "item_index": index}
            # Add potentially useful fields from item to metadata if they exist
            for key in ["name", "position", "issuer", "institution", "organization"]:
                if value := item.get(key):
                    metadata[f"id_{key}"] = value

            all_documents.append(item_doc)
            all_ids.append(item_id)
            all_metadatas.append(metadata)
            processed_count += 1

    print(f"Prepared {processed_count} documents for embedding.")
    return all_documents, all_ids, all_metadatas


# --- 3. Execute Preparation ---
embedding_documents = []
embedding_ids = []
embedding_metadatas = []

embedding_documents, embedding_ids, embedding_metadatas = prepare_embedding_data(
    cv_data=structured_cv_data, sections_to_include=CV_SECTIONS_TO_FOCUS
)

if embedding_documents:
    print("\n--- Sample Prepared Data (First Item) ---")
    print(f"ID        : {embedding_ids[0]}")
    print(f"Metadata  : {embedding_metadatas[0]}")
    print(f"Doc Snippet:\n---\n{embedding_documents[0][:200]}...")
    print("---")

### 5.2. Define Embedding Function

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
import google.generativeai as genai
from google.genai import types


# Use the original class structure provided by the user
class GeminiEmbeddingFunction(EmbeddingFunction):
    """Custom ChromaDB embedding function using the original implementation structure."""

    def __init__(self, document_mode: bool = True) -> None:
        """Initializes based on document_mode, uses model from config."""
        self.embedding_task: str = (
            "retrieval_document" if document_mode else "retrieval_query"
        )
        self.model: str = EMBEDDING_MODEL_NAME

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        """Generates embeddings using client.models.embed_content."""
        response = client.models.embed_content(
            model=self.model,
            contents=input,
            config=types.EmbedContentConfig(task_type=self.embedding_task),
        )
        return [e.values for e in response.embeddings]

### 5.3. Initialize Vector Database

In [ ]:
import chromadb

cv_collection = None  # Initialize variable

try:
    # Initialize persistent client
    print(f"Initializing ChromaDB client at path: {CHROMA_DB_PATH}")
    chroma_client = chromadb.PersistentClient(path=str(CHROMA_DB_PATH))

    gemini_embedder = GeminiEmbeddingFunction(document_mode=True)

    print(f"Accessing collection: '{COLLECTION_NAME}'...")
    cv_collection = chroma_client.get_or_create_collection(
        name=COLLECTION_NAME, embedding_function=gemini_embedder
    )

    print(
        f"Collection '{cv_collection.name}' ready. Item count: {cv_collection.count()}"
    )

except Exception as e:
    print("\nERROR: Failed to initialize ChromaDB client or collection.")
    print(f"Check path ('{CHROMA_DB_PATH}') and ChromaDB setup.")
    print(f"Error details: {e}")
    raise

### 5.4. Generate and Add Embeddings to Collection

In [ ]:
# Add the prepared documents, IDs, and metadata to the ChromaDB collection.
# ChromaDB uses the 'GeminiEmbeddingFunction' provided during collection creation
# to automatically generate embeddings for the documents list.

print(
    f"Adding/updating {len(embedding_documents)} documents in ChromaDB collection"
    f" '{cv_collection.name}'..."
)
try:
    cv_collection.upsert(
        ids=embedding_ids, metadatas=embedding_metadatas, documents=embedding_documents
    )

    print("\nDocuments successfully added/updated in the collection.")
    final_count = cv_collection.count()
    print(f"Collection '{cv_collection.name}' now contains {final_count} items.")
    if final_count < len(embedding_ids):
        print("Warning: Final item count is less than prepared documents count.")

except Exception as e:
    print("\nERROR adding/updating documents in ChromaDB:")
    print(f"Error details: {e}")

## 6. Retrieve Relevant CV Items (RAG)

In [ ]:
# Perform RAG query using the job description

retrieved_ids = []
retrieved_documents = []
retrieved_metadatas = []
retrieved_distances = []

try:
    # Basic validation of inputs from previous steps
    if not JOB_DESCRIPTION:
        raise ValueError("JOB_DESCRIPTION variable is empty.")

    print(f"Performing RAG query on collection '{cv_collection.name}'...")

    # Instantiate embedder for the 'retrieval_query' task type
    query_embedder = GeminiEmbeddingFunction(document_mode=False)

    print("Embedding job description (query)...")
    query_embedding = query_embedder([JOB_DESCRIPTION])[0]

    num_results_to_fetch = MAX_RELEVANT_ITEMS + RETRIEVAL_WINDOW
    print(f"Querying for {num_results_to_fetch} items...")

    results = cv_collection.query(
        query_embeddings=[query_embedding],
        n_results=num_results_to_fetch,
        include=["documents", "metadatas", "distances"],
    )

    # Safely extract results
    retrieved_ids = results.get("ids", [[]])[0]
    retrieved_documents = results.get("documents", [[]])[0]
    retrieved_metadatas = results.get("metadatas", [[]])[0]
    retrieved_distances = results.get("distances", [[]])[0]

    print(f"RAG retrieval complete. Found {len(retrieved_ids)} items.")

except Exception as e:
    print(f"\nERROR during RAG retrieval: {type(e).__name__} - {e}")
    # Ensure lists are reset on error
    retrieved_ids, retrieved_documents, retrieved_metadatas, retrieved_distances = (
        [],
        [],
        [],
        [],
    )

if not retrieved_ids:
    print("\nWarning: No relevant items were retrieved.")

## 7. Rewrite CV Items using LLM Chat

### 7.1. Define Prompts for Rewriting (EN & ES)

In [ ]:
# Structure to hold multi-language prompt components
content = {"en": {}, "es": {}}

In [ ]:
# Example format for the LLM to follow when presenting a modified experience
content["en"]["example"] = """
## Position Name / Title: [JOB TITLE]
- Company Name / Business Name: [COMPANY/ORG NAME]
- Industry type: [INDUSTRY]
- Job Field: [FIELD]
- Sub-Area of Work: [SUB-AREA]

### Original Description

[ORIGINAL DESCRIPTION TEXT]

### Modified description

[SHORT SUMMARY/LEAD-IN]
- [ATS-Optimized achievement/responsibility 1 incorporating keywords]
- [ATS-Optimized achievement/responsibility 2 incorporating keywords]
- ...

### Changes made

- Keywords used: [KEYWORD 1], [KEYWORD 2], ...
- Explanation of changes: [Brief summary of additions/removals/focus shifts].


Shall we continue [Y/n]?
"""

content["es"]["example"] = """
## Nombre del puesto / Título: [TÍTULO PUESTO]
- Nombre de empresa / Negocio: [NOMBRE EMPRESA/ORG]
- Tipo de industria: [INDUSTRIA]
- Área de trabajo: [ÁREA]
- Subárea de trabajo: [SUB-ÁREA]

### Descripción original

[TEXTO DESCRIPCIÓN ORIGINAL]

### Descripción modificada

[RESUMEN CORTO/INTRODUCCIÓN]
- [Logro/responsabilidad optimizado para ATS 1 incorporando palabras clave]
- [Logro/responsabilidad optimizado para ATS 2 incorporando palabras clave]
- ...

### Changes made

- Palabras clave utilizadas: [PALABRA CLAVE 1], [PALABRA CLAVE 2], ...
- Explicación de los cambios: [Resumen breve de adiciones/eliminaciones/reenfoques].


¿Continuamos? [Y/n]
"""

In [ ]:
# Main prompt defining the LLM's role and the interactive workflow.
# Placeholders {n_max_exp}, {example}, {description}, {experiences} will be formatted later.

content["en"]["prompt"] = r"""
You are an expert CV writer specialized in optimizing resumes for Applicant Tracking Systems (ATS) and identifying keywords from job descriptions.
Analyze the provided job description and the list of retrieved CV experiences below. Your goal is to refine the experiences to align strongly with the job description keywords, making them ATS-compatible.

**Output Requirements:**
- ATS-friendly language.
- Concise and impactful wording.
- Focus on achievements and quantifiable results where possible.
- Use active voice (e.g., Developed, Managed, Led).
- Adhere strictly to the requested formats. Avoid extra conversational text.

**Initial Analysis Task:**
1. Identify and list the most critical keywords from the **JOB DESCRIPTION**.
2. List the top {n_max_exp} most relevant **HIGHLIGHTED EXPERIENCES** retrieved, ordered by relevance (most relevant first). Include start/end dates (if available), title/position, and company/organization name.
3. List any remaining **NON-SELECTED EXPERIENCES** retrieved similarly.

**Initial Analysis Output Format:**
**KEYWORDS:** [KEYWORD 1], [KEYWORD 2], ...
**HIGHLIGHTED EXPERIENCES:**
- ([START DATE] - [END DATE]) [JOB TITLE/PROJECT NAME 1], [COMPANY/ORG 1]
- ... (up to {n_max_exp} items) ...
**NON-SELECTED EXPERIENCES:**
- ([START DATE] - [END DATE]) [JOB TITLE/PROJECT NAME X], [COMPANY/ORG X]
- ... (remaining items) ...

**Interaction Flow:**
After presenting the initial analysis, ASK THE FOLLOWING QUESTION EXACTLY:
"Do you possess skills relevant to the keywords? Should any specific HIGHLIGHTED experience be modified?"

WAIT for my response. I will tell you which keywords are most relevant and which specific HIGHLIGHTED experience (by its title/position) I want to modify first.

**Modification Task (Perform ONLY AFTER I select an experience):**
When I ask you to modify a specific experience:
1. Focus ONLY on the single experience I selected.
2. Rewrite its description/highlights to incorporate the relevant **KEYWORDS** identified earlier.
3. Make the description achievement-oriented and ATS-friendly.
4. Present the modified experience using the **EXACT** format shown in the example below. Include the original description text for comparison. Mention the keywords used and explain the changes made.

**Modification Output Format Example:** {example}

**IMPORTANT:** Modify and present ONLY ONE experience at a time, based on my selection. After presenting a modified experience and asking "Shall we continue [Y/n]?", WAIT for my confirmation ('Y' or 'y') before proceeding to ask which *next* highlighted experience I want to modify, OR wait for my feedback/request for further changes on the *current* one. If I enter 'n' or 'q', stop the process.

**Context:**
JOB DESCRIPTION: "{description}"
RETRIEVED CV EXPERIENCES:
{experiences}

Start by performing the **Initial Analysis Task**.
"""

content["es"]["prompt"] = r"""
Eres un experto redactor de CVs, especialista en optimizar para Applicant Tracking Systems (ATS) e identificar palabras clave en descripciones de trabajo.
Analiza la descripción del puesto y la lista de experiencias recuperadas del CV que se proporcionan a continuación. Tu objetivo es refinar las experiencias para alinearlas fuertemente con las palabras clave de la descripción del puesto, haciéndolas compatibles con ATS.

**Requisitos de Salida:**
- Lenguaje amigable para ATS.
- Redacción concisa e impactante.
- Enfoque en logros y resultados cuantificables siempre que sea posible.
- Usar voz activa (ej., Desarrollé, Gestioné, Lideré).
- Adherirse estrictamente a los formatos solicitados. Evitar texto conversacional extra.

**Tarea de Análisis Inicial:**
1. Identifica y lista las palabras clave más críticas de la **DESCRIPCIÓN DE LA OFERTA**.
2. Lista las {n_max_exp} **EXPERIENCIAS DESTACADAS** más relevantes recuperadas, ordenadas por relevance (la más relevante primero). Incluye fechas de inicio/fin (si están disponibles), título/puesto y nombre de la empresa/organización.
3. Lista cualquier **EXPERIENCIA NO SELECCIONADA** restante recuperada de manera similar.

**Formato de Salida del Análisis Inicial:**
**PALABRAS CLAVES:** [PALABRA CLAVE 1], [PALABRA CLAVE 2], ...
**EXPERIENCIAS DESTACADAS:**
- ([FECHA INICIO] - [FECHA FIN]) [TÍTULO PUESTO/PROYECTO 1], [EMPRESA/ORG 1]
- ... (hasta {n_max_exp} ítems) ...
**EXPERIENCIAS NO SELECCIONADAS:**
- ([FECHA INICIO] - [FECHA FIN]) [TÍTULO PUESTO/PROYECTO X], [EMPRESA/ORG X]
- ... (ítems restantes) ...

**Flujo de Interacción:**
Después de presentar el análisis inicial, HAZ LA SIGUIENTE PREGUNTA EXACTAMENTE:
"¿Posees habilidades relevantes para las palabras claves? ¿Debería modificarse alguna EXPERIENCIA DESTACADA específica?"

ESPERA mi respuesta. Te indicaré qué palabras clave son más relevantes y qué EXPERIENCIA DESTACADA específica (por su título/puesto) quiero modificar primero.

**Tarea de Modificación (Realizar SÓLO DESPUÉS de que yo seleccione una experiencia):**
Cuando te pida modificar una experiencia específica:
1. Enfócate SÓLO en la única experiencia que seleccioné.
2. Reescribe su descripción/logros para incorporar las **PALABRAS CLAVES** relevantes identificadas anteriormente.
3. Haz la descripción orientada a logros y amigable para ATS.
4. Presenta la experiencia modificada usando el formato **EXACTO** mostrado en el ejemplo a continuación. Incluye el texto de la descripción original para comparación. Menciona las palabras clave usadas y explica los cambios realizados.

**Ejemplo de Formato de Salida de Modificación:** {example}

**IMPORTANTE:** Modifica y presenta SÓLO UNA experiencia a la vez, según mi selección. Después de presentar una experiencia modificada y preguntar "¿Continuamos? [Y/n]?", ESPERA mi confirmación ('Y', 'y' o Enter) antes de preguntar qué *siguiente* experiencia destacada quiero modificar, O espera mi feedback/solicitud de más cambios en la *actual*. Si ingreso 'n' o 'q', detén el proceso.

**Contexto:**
DESCRIPCIÓN DE LA OFERTA: "{description}"
EXPERIENCIAS DEL CV RECUPERADAS:
{experiences}

Comienza realizando la **Tarea de Análisis Inicial**.
"""

# Note: The placeholders {n_max_exp}, {example}, {description}, {experiences}
# will be filled using .format() before sending the prompt to the LLM.

In [ ]:
# Messages for user interaction

# --- quit message ---
content["en"]["quit_msg"] = "To exit, enter 'q' or 'quit'"
content["es"]["quit_msg"] = "Para salir, ingresa 'q' o 'salir'"

# --- initial response ---
content["en"]["initial_response"] = "LLM Initial Analysis Response:"
content["es"]["initial_response"] = "Respuesta de Análisis Inicial del LLM:"

# --- exit message ---
content["en"]["exit_msg"] = "\nExiting interactive loop."
content["es"]["exit_msg"] = "\nSaliendo del bucle interactivo."

# --- y interpretation ---
content["en"][
    "y_interpretation"
] = "(Interpreting empty input as 'Y' to continue/confirm)"
content["es"][
    "y_interpretation"
] = "(Interpretando entrada vacía como 'Y' para continuar/confirmar)"

# --- send message ---
content["en"]["send_message"] = "\nSending message to LLM..."
content["es"]["send_message"] = "\nEnviando mensaje al LLM..."

# --- received message ---
content["en"]["received_message"] = "LLM response received:"
content["es"]["received_message"] = "Respuesta del LLM recibida:"

# --- Keyboard Interrupt ---
content["en"]["keyboard_interrupt"] = "\nLoop interrupted by user (Ctrl+C). Exiting."

### 7.2. Initiate Chat and Generate Initial Response

In [ ]:
from IPython.display import Markdown

# --- 1. Format Retrieved Experiences for Prompt ---
formatted_experiences = []
if "retrieved_documents" in locals() and retrieved_documents:
    for i, doc in enumerate(retrieved_documents):
        meta = retrieved_metadatas[i] if i < len(retrieved_metadatas) else {}
        section = meta.get("section", "Unknown Section")
        name = meta.get("id_name", "")
        position = meta.get("id_position", "")
        header = f"{name}{(' - ' + position) if position else ''} ({section})"
        formatted_experiences.append(f"--- Experience {i+1}: {header} ---\n{doc}")
    experiences_context_string = "\n\n".join(formatted_experiences)
else:
    experiences_context_string = (
        "No relevant experiences were retrieved from the database."
    )
    print("Warning: No retrieved experiences to include in the prompt.")

# --- 2. Select Language and Format Final Prompt ---
content_lang = content[LANGUAGE]  # Select EN or ES content dict
initial_prompt = content_lang["prompt"].format(
    n_max_exp=MAX_RELEVANT_ITEMS,
    example=content_lang["example"],
    description=JOB_DESCRIPTION,
    experiences=experiences_context_string,
)

# --- 3. Initialize LLM and Chat for Rewriting ---
chat = None  # Initialize chat variable
try:
    rewriting_model = GENERATIVE_MODEL_NAME

    rewriting_generation_config = {
        "temperature": REWRITING_LLM_TEMPERATURE,
        "top_p": REWRITING_LLM_TOP_P,
        "top_k": REWRITING_LLM_TOP_K,
    }

    # Start a new chat session (history is empty)
    chat = client.chats.create(
        model=GENERATIVE_MODEL_NAME, history=[], config=rewriting_generation_config
    )
    print(f"Chat session initialized with model '{GENERATIVE_MODEL_NAME}'.")

    # --- 4. Send Initial Prompt and Get Response ---
    print("Sending initial prompt to LLM for analysis...")
    # Send the formatted prompt, applying the specific generation config
    initial_response = chat.send_message(initial_prompt)


except NameError as e:
    # Catch errors if genai or types wasn't imported/available
    print(
        "ERROR: Required object not defined (e.g., 'genai', 'types'). Check cell"
        " execution order."
    )
    print(f"Error: {e}")
    chat = None
except Exception as e:
    print("\nERROR initializing chat or sending initial message:")
    print(f"Error Type: {type(e).__name__}")
    print(f"Error details: {e}")
    chat = None

# Ensure variables exist for the next step
if "initial_response" not in locals():
    initial_response = None

# Verification print moved to focus only on successful execution of this cell's task
if chat and initial_response:
    print("Initial LLM response obtained successfully.")
else:
    print("Failed to obtain initial LLM response or initialize chat.")
    raise RuntimeError("Chat session or initial response is not valid.")

### 7.3. Interactive Refinement Loop

The following cell starts the interactive loop. First, it displays the initial analysis received from the LLM in the previous step. Then, it repeatedly prompts you for input.

Based on the LLM's analysis and questions, provide your feedback or select an experience to modify. Your input will be sent back to the LLM, and its response will be displayed.

Enter 'q' or 'quit' (or the Spanish equivalent if configured) to exit the loop.

In [ ]:
from IPython.display import Markdown, display

content_lang = content[LANGUAGE]  # Select EN or ES content dict

# --- 1. Display Initial LLM Analysis ---
print("\n" + "=" * 80)
print(content_lang["initial_response"])
print("=" * 80)
display(Markdown(initial_response.text))
print("=" * 80)

# --- 2. Interactive Refinement Loop ---
quit_msg = content_lang.get("quit_msg", "Enter 'q' or 'quit' to exit")
# Define various inputs that signal exiting the loop (incl. 'n' for No)
quit_options = {"q", "quit", "exit", "salir", "n"}

while True:
    # Display quit instructions clearly
    print("\n" + f" {quit_msg} ".center(80, "-"))
    try:
        # Prompt user for input
        user_input = input("> ")

        # Check if the user wants to exit
        if user_input.lower() in quit_options:
            print(content_lang["exit_msg"])
            break

        # Assume 'Yes' (continue/approve) if user just presses Enter
        if not user_input:
            user_input = "Y"
            print(content_lang["y_interpretation"])

        # Send the user's message to the LLM via the chat session
        print(content_lang["send_message"])
        llm_response = chat.send_message(user_input)
        print(content_lang["received_message"])

        # Display the LLM's response using Markdown rendering
        display(Markdown(llm_response.text))

    except KeyboardInterrupt:
        print(content_lang["keyboard_interrupt"])
        break
    except Exception as e:
        # Catch potential errors during chat interaction
        print("\nERROR during interactive chat:")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error details: {e}")
        print("Exiting interactive loop due to error.")
        break  # Exit loop on error

## 8. Conclusion & Take-aways

This notebook demonstrated a complete workflow for optimizing CV sections against a job description using RAG and Generative AI:

* **CV Structuring:** Successfully parsed a PDF CV into a structured JSON format using an LLM (Gemini).
* **Vector Search (RAG):** Created embeddings for CV items and utilized ChromaDB to retrieve the sections most semantically relevant to a specific job description.
* **LLM-Powered Rewriting:** Employed a Gemini chat model in an interactive loop to refine the retrieved CV sections, incorporating keywords and optimizing for ATS.
* **Self-Contained Workflow:** The entire process, from PDF loading to interactive rewriting, is contained within this single Kaggle notebook.

This approach allows for targeted CV customization, potentially improving application success rates by better aligning the CV with job requirements and ATS filtering.

## 9. Final Remarks / Contact

🌟 CONGRATULATIONS 🌟

If you’ve made it all the way to the end of this notebook (whether you’re an evaluator, a fellow data‑enthusiast, or just curious about the process), you definitely deserve a prize!

Our project is evolving every week, and your feedback keeps it alive.
👉 Show some love by giving the repo a star on GitHub:
https://github.com/framunoz/cv-analyser-with-rag/

A single ⭐ helps the framework grow, motivates new features, and lets us know this work is valuable to the community.

Thanks for reading, building, and experimenting alongside us—see you in the next commit!